<table>
<table class="ee-notebook-buttons" align="left">
   <td><img width=120px src="https://github.com/modoukebe/depot00-shape/blob/main/sodagri_logo.jpg?raw=1" /></td>
<table class="ee-notebook-buttons" align="right">
   <td> <img width=120px src="https://github.com/modoukebe/depot00-shape/blob/main/LTA.jpg?raw=1"></td>
<font size="+2.5" color=pink><b> <center><u> PLATEFORME DE SUIVI DES CULTURES DE RIZ </u></center></b></font>
</table>

In [2]:
# Check geemap installation
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [3]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import Map, basemaps, WidgetControl

In [4]:
import ee
from google.oauth2.credentials import Credentials
user_secret = "earth_engine" # Your user secret, defined in the add-on menu of the notebook editor
refresh_token = "1//05vucBuiE2mIhCgYIARAAGAUSNwF-L9Ir212YgWEvqVX54eoEETXtaXpO8b3bUjYzN25XJziKStXqx7H_3_KgXNLnyni3u5Bx2fk"
credentials = Credentials(
       None,
       refresh_token=refresh_token,
       token_uri=ee.oauth.TOKEN_URI,
       client_id=ee.oauth.CLIENT_ID,
       client_secret=ee.oauth.CLIENT_SECRET,
       scopes=ee.oauth.SCOPES)
ee.Initialize(credentials=credentials)

In [5]:
Map = geemap.Map(center=[14.499454, -14.445561499999997], zoom=8, add_google_map=False)

reg = 'sn_region_pol_gcs.shp'
regfc = geemap.shp_to_ee(reg)
Map.addLayer(regfc, {}, 'REGION')

dep = 'sn_departmt_pol_gcs.shp'
states = geemap.shp_to_ee(dep)
Map.addLayer(states, {}, 'DEPARTEMENT')

com = 'sn_com_rurale_pol_gcs.shp'
fc = geemap.shp_to_ee(com)
Map.addLayer(fc, {}, 'COMMUNE')

am = 'Aménagements.shp'
amfc = geemap.shp_to_ee(am)
Map.addLayer(amfc, {}, 'AMENAGEMENTS')


Map

Map(center=[14.499454, -14.445561499999997], controls=(WidgetControl(options=['position'], widget=HBox(childre…

In [6]:
# Designe interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

admin1_widget = widgets.Text(
    description='région:',
    value='DAkar',
    width=200,
    style=style
)

admin2_widget = widgets.Text(
    description='département:',
    value='Dakar',
    width=200,
    style=style
)

admin3_widget = widgets.Text(
    description='commune:',
    value='Dakar',
    width=200,
    style=style
)

admin4_widget = widgets.Text(
    description='Aménagement:',
    value='10',
    width=300,
    style=style
)

band_combo = widgets.Dropdown(
    description='Champs:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style
)

nd_options = ['Vegetation Index (NDVI)', 
              'Water Index (NDWI)',
              'Modified Water Index (MNDWI)',
              'Snow Index (NDSI)',
              'Soil Index (NDSI)',
              'Burn Ratio (NBR)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Modified Water Index (MNDWI)', description='Indices:', style=style)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([admin1_widget, admin2_widget, admin3_widget, admin4_widget]),
    widgets.HBox([band_combo, nd_indices]),
    submit
])

full_widget

In [7]:
# Capture user interaction with the map

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)
        
        with output_widget:
            output_widget.clear_output()
        
            try:
                feature = selected_fc.first()
                admin1_id = feature.get('Region').getInfo()
                
                admin2_id = feature.get('Departemen').getInfo()
                admin3_id = feature.get('Nomcollect').getInfo()
                admin4_id = feature.get('OREG_1').getInfo()
                
                #admin1_fc = ee.Feature(states.filter(ee.Filter.eq('STATEFP', statefp)).first())                
                #admin1_id = admin1_fc.get('NAME').getInfo()
                admin1_widget.value = admin1_id
                admin2_widget.value = admin2_id
                admin3_widget.value = admin3_id
                admin4_widget.value = admin4_id
                Map.layers = Map.layers[:4]        
                geom = selected_fc.geometry()
                layer_name = admin1_id + '-' + admin2_id + '-' + admin3_id + '-' + admin4_id
                Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
                print(layer_name)
            except:
                print('No feature could be found')
                Map.layers = Map.layers[:4]        
            
        Map.default_style = {'cursor': 'pointer'}
    else:
        Map.draw_count = 0

Map.on_interaction(handle_interaction)